

# **Synchronization in Complex Networks and Applications**


---





Victor Hugo F. Francheto

University of São Paulo, SP

victor.francheto@usp.br



Bruno B. Pereira

University of São Paulo, SP

bruno.pereira@usp.br


---



**Tema escolhido:**

### **5 - Como a sincronização é inPuenciada pela topologia da rede?**

Ver: https://github.com/fabridamicelli/kuramoto

Use o artigo:
Synchronization in complex networks
https://www.sciencedirect.com/science/ar;cle/pii/S0370157308003384

Gerar redes usando diferentes modelos e verificar como a sincronização muda de acordo com o
modelo. Pode ser gerado um gráfico do parâmetro de ordem (r) em função do acomplamento
lambda.

Analisar como a estrutura de comunidades afeta a sincronização, mudando o parâmetro mu no
modelo de LFR que gera redes modulares.

---

Iniciamos importando os pacotes basicos para o estudo.

In [1]:
from pylab import *
import numpy as np
from numpy import linalg as LA
from scipy.integrate import *
import matplotlib.pyplot as plt
from scipy.special import zeta
import networkx as nx
import community.community_louvain as community_louvain

Para estudar a sincronização em diferentes topologias, utilizaremos o modelo de Kuramoto.

In [2]:
# First-order Kuramoto model
def F(Y,t, G, W, lbd):
    #global W, N, lbd, G
    N = len(G.nodes())
    F = zeros((N),float)
    S = zeros((N),float)
    dg = G.degree()
    for i in range(0,N):
        for j in G.neighbors(i):
            S[i] = S[i] + sin(Y[j]-Y[i]) #
    ld = lbd
    F = W + ld*S #traditional Kuramoto
    return F

Aqui estamos definindo alguns paramentros das redes que analizaremos a sincronização.

In [3]:
#>>Parametros de Rede<<#

#Número de nós
N = 100

# Grau médio
av_degree = 15

# Probabilidade de conexão para Modelos Erdős–Rényi
p = float(av_degree)/float(N)

# Número de nós incluídos a cada passo de tempo no modelo Barabasi Albert e Waxman
m = int(av_degree/2)

# Número de vizinhos no modelo Small Words
kappa = av_degree

#>>Paremetros de redes com comunidades<<#

# Definir tamanhos das partições (comunidades)
partition_sizes = [30, 60, 10]

# Probabilidade de aresta dentro da mesma comunidade
p_in = 0.5

# Probabilidade de aresta entre comunidades diferentes
p_out = 0.05

seed = 42


#>>Paremetros de redes com comunidades LFV<<#
tau1 = 3
tau2 = 1.5
mu = 0.1
minc = 32
maxc = 32

In [4]:

def rede_com_comunidades(partition_sizes,p_in,p_out,seed):

  # Definir tamanhos das partições (comunidades)
  partition_sizes = partition_sizes

  # Definir probabilidades de conexão
  p_in = p_in  # Probabilidade de aresta dentro da mesma comunidade
  p_out = p_out # Probabilidade de aresta entre comunidades diferentes

  # Gerar o grafo particionado aleatório
  G = nx.random_partition_graph(partition_sizes, p_in, p_out, seed=seed)
  return G




In [ ]:
def rede_com_comunidades_LFR(N, tau1, tau2, mu, k, minc, maxc)
  N = N
  tau1 = tau1
  tau2 = tau2
  mu = mu
  k = k
  minc = minc
  maxc = maxc
  G = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = mu, min_degree = k,
                          max_degree = k, min_community=minc, max_community = maxc, seed = 10)
  return G

In [5]:
def rede_eros_renie(N,p,seed):
  N = N
  p = p
  seed = seed
  G = nx.gnp_random_graph(N, p, seed=seed, directed=False)
  return G



In [6]:
def rede_small_word(N, k, p, seed):
  N = N
  k = k
  p = p
  seed = seed
  k = int(k)
  G = nx.watts_strogatz_graph(N, k, p, seed=seed)
  return G


In [7]:
def rede_barabasi_albert(N,m):
   m = m
   G = nx.barabasi_albert_graph(N, m)
   return G

In [8]:
def rede_waxman(N,m):
   m = m
   G = nx.barabasi_albert_graph(N, m)
   return G

In [9]:
#G = nx.Graph()
#G = Rede_Eros_Renie(N,p,seed)
#G.remove_edges_from(nx.selfloop_edges(G))
#Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
#G = G.subgraph(Gcc[0])
#G = nx.convert_node_labels_to_integers(G, first_label=0)
#pos = nx.spring_layout(G)
#nx.draw(G, pos, node_color="c", node_size=10, with_labels=False)

In [10]:

def calculando_sincronizacao(G):
  G = G.to_undirected() # Remevendo direção da Rede
  G = nx.convert_node_labels_to_integers(G, first_label=0) # Convertendo os Labels em inteiros
  N = len(G) # Número de Nós

  vk = dict(G.degree()) # Grau
  vk = list(vk.values()) # ??
  av_k = mean(vk) # Grau Médio

  A = nx.adjacency_matrix(G) #Criando matriz de adjacencia
  A = A.todense() #ajustando a matrix
  eig, vectors = LA.eig(A) # Calculando auto valores
  lambda_max = eig.max() # Obtendo o maior auto valor

  #>>Initial phases<<#
  w0 = np.pi
  y0 = np.random.normal(0, w0, size = N)

  # Frequência Natural
  W = np.random.uniform(-w0, w0, size = N)

  # Acoplamento Crítico
  hist, bin_edges = np.histogram(W, density=True)
  g0 = max(hist)
  lambda_c = 2/(np.pi*lambda_max*g0)

  # Tempo de Integração
  t0 = 0.0  # Inicio
  h = 0.02 # Incremento
  Np = 3000 # Numero de interações
  tStop = int(Np*h) # Fim da Integração
  t = linspace(t0,tStop,Np)

  #>>Processo de Sincronização<<#
  vlbd = []  # vetor que armazena a força de acoplamento
  aux = 0
  Ns = 1 # Numero de Simulações
  vlbd = np.arange(0, 10*lambda_c, lambda_c/5) #variation of the coupling strength
  mvr = np.zeros(shape=shape(vlbd))
  for s in arange(0,Ns):
      vr = []  # vector that stores the order parameter
      for lbd in vlbd:
          y = odeint(F, y0, t, (G, W, lbd))
          y = np.array(y)
          y = y[range(len(y)-round(0.9*len(y)),len(y)),:]
          y = y.mean(0)
          psi = mean(y)
          r = abs((1.0/N)*sum(np.exp(1j*y)))
          vr.append(r)
          aux = aux + 1
          #print(aux, ": ", lbd, r)
          y0 = y # adiabatic increase
      mvr = mvr + vr
  mvr = mvr/Ns
  vlbd = array(vlbd, dtype=float)
  return vlbd,mvr,lambda_c,




In [11]:
G_1 = nx.Graph()
G_1 = rede_com_comunidades(partition_sizes,p_in,p_out,seed)
vlbd_1,mvr,lambda_c = calculando_sincronizacao(G_1)

G_2 = nx.Graph()
G_2 = rede_eros_renie(N,p,seed)
vlbd_2,mvr,lambda_c = calculando_sincronizacao(G_2)

G_3 = nx.Graph()
G_3 = rede_small_word(N, kappa, p, seed)
vlbd_3,mvr,lambda_c = calculando_sincronizacao(G_3)

G_4 = nx.Graph()
G_4 = rede_barabasi_albert(N,m)
vlbd_4,mvr,lambda_c = calculando_sincronizacao(G_4)

G_5 = nx.Graph()
G_5 = rede_waxman(N,m)
vlbd_5,mvr,lambda_c = calculando_sincronizacao(G_5)

G_6 = nx.Graph()
G_6 = rede_com_comunidades_LFR(N, tau1, tau2, mu, kappa, minc, maxc)
vlbd_5,mvr,lambda_c = calculando_sincronizacao(G_6)






In [12]:
# show the results

plt.figure(figsize=(50,5))
plt.plot(vlbd_1, mvr, '-o', color='red', linewidth=1 ) # Com Comunidades
plt.plot(vlbd_2, mvr, '-o', color='blue', linewidth=1 ) # ER
plt.plot(vlbd_3, mvr, '-o', color='green', linewidth=1 ) # SW
plt.plot(vlbd_4, mvr, '-o', color='pink', linewidth=1 ) # BA
plt.plot(vlbd_5, mvr, '-o', color='gray', linewidth=1 ) # WX
plt.plot(vlbd_6, mvr, '-o', color='black', linewidth=1 ) # LFR





plt.axvline(x =lambda_c)
xlabel(r"$\lambda$")
ylabel("r")
#plt.legend(True)
plt.plot(True)